# Define and assign bondwire profiles

## Imports

In [1]:
import os
from ansys.edb.database import Database
from ansys.edb.layout.cell import Cell, CellType
from ansys.edb.layer import LayerType, StackupLayer
from ansys.edb.net import Net
from ansys.edb.hierarchy import CellInstance
from ansys.edb.definition import ApdBondwireDef
from ansys.edb.primitive import Bondwire, BondwireType, Rectangle, RectangleRepresentationType, BondwireCrossSectionType
from ansys.edb.session import launch_session

## Configs

In [2]:
# RPC_SERVER = None if separate server is already running, Root dir of server executable otherwise.
RPC_SERVER = "C:\\ansysDev\\view1\\build_output\\64Release"
ROOT = os.getcwd()
EDB_FILE = os.path.join(ROOT, "define_and_assign_bondwire_profiles.aedb")

## Launch a session

In [3]:
# Make sure old one has been disconnected prior to starting a new one, if any
session = launch_session(RPC_SERVER, 50051)

## Basic setup of empty EDB

In [4]:
"""Create Database, Cell, Layout, Net"""
db = Database.create(EDB_FILE)

#cells
cell = Cell.create(db, CellType.CIRCUIT_CELL, "Design")
cell_start = Cell.create(db, CellType.CIRCUIT_CELL, "Left_Rect")
cell_end = Cell.create(db, CellType.CIRCUIT_CELL, "Right_Rect")

# Layouts 
layout = cell.layout
layout_start = cell_start.layout
layout_end = cell_end.layout

# Net
net = Net.create(layout, 'GND')
net_start = Net.create(layout_start, 'GNDs')
net_end = Net.create(layout_end, 'GNDe')

id: 1



## Setup Layers for the different cells

In [5]:
"""Create layers, add them to layer collection and retrieve them again."""

# Layers in layout's layer collection
layer_gnd = StackupLayer.create('Ground', LayerType.SIGNAL_LAYER, 0.0001, 0, "copper")
layer_start = StackupLayer.create('Layer1', LayerType.SIGNAL_LAYER, 0.003, 0.0001, "copper")
layer_end = StackupLayer.create('Layer2', LayerType.SIGNAL_LAYER, 0.003, 0.0031, "copper")
lc = layout.layer_collection
lc.add_layers([layer_gnd, layer_start, layer_end])

# Layers for additional cells
layer_inner_start = StackupLayer.create('Start', LayerType.SIGNAL_LAYER, 0.0001, 0, "copper")
layer_inner_end = StackupLayer.create('End', LayerType.SIGNAL_LAYER, 0.0001, 0, "copper")
lcs = layout_start.layer_collection
lcs.add_layers([layer_inner_start])
lce = layout_end.layer_collection
lce.add_layers([layer_inner_end])

# Retrieve layers collection and layers
layer_gnd = lc.find_by_name('Ground')
layer_start = lc.find_by_name('Layer1')
layer_end = lc.find_by_name('Layer2')
layer_inner_start = lcs.find_by_name('Start')
layer_inner_end = lce.find_by_name('End')

# Priint layers created for lc
print("LC", [(l.name) for l in lc.get_layers() ])
# Priint layers created for lcs
print("LCS", [(l.name) for l in lcs.get_layers() ])
# Priint layers created for lcs
print("LCE", [(l.name) for l in lce.get_layers() ])


print(layer_inner_start.name,layer_inner_end.name)


LC ['Layer2', 'Layer1', 'Ground']
LCS ['Start']
LCE ['End']
Start End


## Create geometry

In [6]:
"""Create Primitive objects"""

# Rectangles
um = lambda val : val * 1e-6
rect = Rectangle.create(layout=layout_start,
                        layer=layer_inner_start,
                        net=net_start,
                        rep_type=RectangleRepresentationType.LOWER_LEFT_UPPER_RIGHT,
                        param1=um(0.0),
                        param2=um(0.0),
                        param3=um(30),
                        param4=um(30),
                        corner_rad=0.0,
                        rotation=0.0
                        )

rect_2 = Rectangle.create(layout=layout_end,
                          layer=layer_inner_end,
                          net=net_end,
                          rep_type=RectangleRepresentationType.LOWER_LEFT_UPPER_RIGHT,
                          param1=um(0.0),
                          param2=um(0.0),
                          param3=um(20),
                          param4=um(20),
                          corner_rad=0.0,
                          rotation=0.0
                          )
#Retrieve the primitives
primitives_created = layout_start.primitives + layout_end.primitives

# Print primitives created and their layers
print([(p.primitive_type.name, p.layer.name) for p in primitives_created ])

[('RECTANGLE', 'Start'), ('RECTANGLE', 'End')]


## Create the cell instances

In [7]:
cis = CellInstance.create(layout, 'CellInstanceStart', layout_start)
cie = CellInstance.create(layout, 'CellInstanceEnd', layout_end)
cie.location=(um(60),um(10))

## Create the bondwire definition

In [8]:
# Create an APD Bondwire Definition
wire_0 = ApdBondwireDef.create(db, 'Wire_0')

# Bondwire Parameters name (nm) diameter(dia) and material(mat) should be set as strings(quoted)
parameters_str = "bwd(nm=\'Wire_0\', ven=true, for=true, dia=\'2.54e-05\', mat=\'gold\',\
col=0, vis=true, dih=0, nfc=6, seg(ht=0, hv=0, vt=0, vv=0, ht=0, hv=0, vt=0, vv=0))"

# Set parameters
wire_0.set_parameters(parameters_str)

## Create the bondwire

In [9]:
# Bondwire
bondwire = Bondwire.create(
    layout=layout,
    bondwire_type=BondwireType.APD,
    definition_name='Wire_0',
    placement_layer=layer_gnd.name,
    width=um(5),
    material='gold',
    start_context=cis,
    start_layer_name=layer_inner_start.name,
    start_x=um(15),
    start_y=um(15),
    end_context=cie,
    end_layer_name=layer_inner_end.name,
    end_x=um(55),
    end_y=um(15),
    net=net,
)
bondwire.cross_section_type = BondwireCrossSectionType.RECTANGLE
bondwire.cross_section_height = um(5)
bondwire.set_traj(um(15), um(15), um(70), um(15))

# Save database

In [10]:
"""Save database if you wish."""

db.save()

# Disconnect from database

In [11]:
"""Don't forget to disconnect it!"""

session.disconnect()